In [18]:
import matplotlib.pyplot as plt
import numpy as np

In [19]:
# Make maptlotlib show graphs in a new window
%matplotlib qt 

In [29]:
import numpy as np
import matplotlib.pyplot as plt

class ValueMap:
    def __init__(self, gridSize, southWestPos, cellSize, noDataVal):
        self.gridSize = gridSize
        self.southWestPos = southWestPos
        self.cellSize = cellSize
        self.noDataVal = noDataVal
        
        self.gridData = np.empty((gridSize[0], gridSize[1]))
    
    def setGridData(self, gridData):
        self.gridData = gridData
    
    def determineValue(self, lattitude, longitude):
        yIndex = self.gridData.shape[0] -  int(abs(lattitude - self.southWestPos[0]) / self.cellSize)
        xIndex = int(abs(longitude - self.southWestPos[1]) / self.cellSize)
        if yIndex > 0:
            yIndex = yIndex - 1
        if xIndex > 0:
            xIndex = xIndex - 1
        print([xIndex, yIndex])
        self.gridData[yIndex][xIndex]=25000
        return self.gridData[yIndex][xIndex]
    
    def plot(self, colormap='jet'):
        plt.imshow(self.gridData, cmap=colormap, interpolation='nearest')
        plt.colorbar()
        plt.show()

    def writeToFile(self, filePath):
        with open(filePath, "w+", encoding='utf-8') as outputFile:
            header = [
                f"nrows {self.gridData.shape[0]}",
                f"ncols {self.gridData.shape[1]}",
                f"cellsize {self.cellSize}",
                f"nodata_value {self.noDataVal}",
                f"SOUTHWEST LONGITUDE {self.southWestPos[1]}",
                f"SOUTHWEST LATITUDE {self.southWestPos[0]}"
            ]

            outputFile.writelines([string + '\n' for string in header])

            coordData = []
            rowIndex = 0
            for row in self.gridData:
                thisRow = ""
                for itemIndex, item in enumerate(row):
                    thisRow = thisRow + str(item)
                    if(itemIndex + 1 != len(row)):
                        thisRow = thisRow + " "
                    
                thisRow = thisRow + "\n"
                coordData.append(thisRow)

                rowIndex +=1 
                if(rowIndex % 100 == 0):
                    print(f"Writing line {rowIndex}/{self.gridData.shape[0]}")

            outputFile.writelines(coordData)


In [20]:
dataDate = "10/1/2024" # Date of the data to be collected in format %D/%M/%Y

gridData = []

In [21]:
# path = f"data/solarIradiance/{dataDate.replace('/', '')}.grid"
path = "data/density.grid"
# path = "irradianceVals.grid"
with open(path, "r", encoding="utf-8") as gridFile:
    gridData = gridFile.readlines()
    
for index, line in enumerate(gridData):
    gridData[index] = line.replace("\n", "")

In [22]:
gridSize = [-1,-1]
southWestPos = [-1,1]
cellSize = -1
noDataVal = -1
for line in gridData:
    if "ncols" in line:
        gridSize[1] = int(line[line.find(" ")+1:])
    if "nrows" in line:
        gridSize[0] = int(line[line.find(" ")+1:])
    if "cellsize" in line:
        cellSize = float(line[line.find(" ")+1:])
    if "nodata_value" in line:
        noDataVal = float(line[line.find(" ")+1:])
    if "SOUTHWEST LATITUDE" in line:
        line = line.replace(" ", "")
        southWestPos[0] = float(line[line.find("LATITUDE")+8:])
    if "SOUTHWEST LONGITUDE" in line:
        line = line.replace(" ", "")
        southWestPos[1] = float(line[line.find("LONGITUDE")+9:])

        

In [40]:
# Put grid data into a np array
mapData = np.empty((gridSize[0], gridSize[1]))

rowIndex = 0 
for line in gridData:
    row = line.split(" ")
    try:
        testVal = float(row[0]) # Ensure that the first value is a float, thus only considering values
        for itemIndex, item in enumerate(row):
            dataVal = float(item)
            if dataVal == -3:
                dataVal = 2
            elif (dataVal == noDataVal) or (dataVal < 0):
                mapData[rowIndex][itemIndex]=0
            else:
                mapData[rowIndex][itemIndex]=1
        
        rowIndex = rowIndex + 1
    except ValueError:
        pass



In [41]:
irradianceVals = ValueMap(gridSize, southWestPos, cellSize, noDataVal)
irradianceVals.setGridData(mapData)

In [45]:
irradianceVals.plot("gist_yarg")